In [235]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [236]:
df = pd.read_csv("train_cleaned.csv")

# Observe numeric variables for PCA 

In [237]:
df.select_dtypes(include=[np.number]).head().columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape',
       'LandSlope', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC', 'CentralAir',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces',
       'FireplaceQu', 'GarageYrBlt', 'GarageFinish', 'GarageCars',
       'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'PoolQC', 'Fence', 'MiscVal', 'MoSold', 'YrSold', 'TotalSF',
       'SalePrice'],
      dtype='object')

We will include the TotRmsAbbvGrd even though it may be an ordinal variable the range of the values is quite wide. 
Year built and remodel date can be changed to the age of the building and the number of years it has been remodeled 

In [238]:
import datetime
df["Age"] = datetime.datetime.now().year-df['YearBuilt'] 
df["RemodAge"] = datetime.datetime.now().year-df['YearRemodAdd']

In [239]:
numeric = df[['LotFrontage', 'LotArea','MasVnrArea', 
   'BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF',
   '1stFlrSF', 'LowQualFinSF','GrLivArea','TotRmsAbvGrd',
   'GarageYrBlt','GarageArea','WoodDeckSF','OpenPorchSF',
   'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea',
   'MiscVal','TotalSF','Age','RemodAge']]

#23 numerical predictors

# Standardize Data for PCA 
* For PCA the variables should be centered to have mean zero.
* In addition, the results of PCA will also depend on whether the variables have been individually scaled

In [240]:
from sklearn.preprocessing import StandardScaler
numeric = StandardScaler().fit_transform(numeric)
numeric.shape #(n_samples, n_features/predictors)

(1458, 23)

In [241]:
from sklearn.decomposition import PCA

#Dont set number of components so all components are kept
pca = PCA()
#fit and trasnform pca/ Eigen Decomposition/ dimension reduction
pcahouse = pca.fit_transform(numeric)
pcaloadings = pd.DataFrame(pcahouse.transpose())
pcaloadings18 = pcaloadings.iloc[:, [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]]
pcaloadings18.columns = ["PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10","PC11","PC12","PC13","PC14","PC15","PC16", "PC17", "PC18"]
#pcaloadings15
print (pca.explained_variance_)
print (pca.explained_variance_ratio_)
print (pca.explained_variance_ratio_.cumsum())

[  5.49218951e+00   1.96714091e+00   1.69598740e+00   1.19122664e+00
   1.13891032e+00   1.10729839e+00   1.07407664e+00   1.03147727e+00
   9.97882158e-01   9.77485122e-01   8.96993061e-01   8.79831290e-01
   8.45395020e-01   7.56112698e-01   6.90281544e-01   6.60184051e-01
   6.15394363e-01   3.19281604e-01   3.02379222e-01   2.01794546e-01
   1.58678237e-01   4.77336878e-31   2.94624172e-32]
[  2.38790848e-01   8.55278655e-02   7.37385828e-02   5.17924628e-02
   4.95178402e-02   4.81434080e-02   4.66989844e-02   4.48468376e-02
   4.33861808e-02   4.24993531e-02   3.89996983e-02   3.82535343e-02
   3.67563052e-02   3.28744651e-02   3.00122410e-02   2.87036544e-02
   2.67562767e-02   1.38818089e-02   1.31469227e-02   8.77367591e-03
   6.89905379e-03   2.07537773e-32   1.28097466e-33]
[ 0.23879085  0.32431871  0.3980573   0.44984976  0.4993676   0.54751101
  0.59420999  0.63905683  0.68244301  0.72494236  0.76394206  0.8021956
  0.8389519   0.87182637  0.90183861  0.93054226  0.9572985

# Choosing the number of components 
* based on the elbow of the scree plot
* Kaiser Criterion (Eigenvalues < 1 since we have already scaled the data)
* Or if we just to retain 90% of the variability of the data set

In [242]:
# By observing the variance explained cumsum we see that inorder to keep
# at least 90% of the variability we need to keep at least up to the 15th principal component
print (pca.explained_variance_ratio_.cumsum())

[ 0.23879085  0.32431871  0.3980573   0.44984976  0.4993676   0.54751101
  0.59420999  0.63905683  0.68244301  0.72494236  0.76394206  0.8021956
  0.8389519   0.87182637  0.90183861  0.93054226  0.95729854  0.97118035
  0.98432727  0.99310095  1.          1.          1.        ]


In [243]:
# Principal component loadings 
pcaloadings18['Predictors'] = ['LotFrontage', 'LotArea','MasVnrArea', 'BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF', 'LowQualFinSF','GrLivArea','TotRmsAbvGrd','GarageYrBlt','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','TotalSF','Age','RemodAge']

/Users/chi/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [244]:
# Now we can observe the loadings 
pcaloadings18.set_index("Predictors")


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18
Predictors,,,,,,,,,,,,,,,,,,
LotFrontage,1.917055e-01,3.960037e-01,-1.316549e+00,3.027847e+00,-6.125670e-01,3.016260e+00,1.320555e+00,-1.084522e+00,-2.414237e+00,-1.563405e+00,3.253526e+00,-1.946871e+00,2.676080e+00,-3.278346e-01,-1.895488e+00,-1.249326e+00,-2.419240e+00,7.546319e-02
LotArea,-1.058227e+00,-1.033833e+00,2.478953e+00,-3.551621e-01,-1.754340e+00,-1.615046e+00,1.184817e+00,2.743441e+00,-3.275327e-01,-9.285840e-01,-2.557448e-01,-9.083198e-01,-3.823285e-01,7.655950e-01,-4.903892e-01,-8.176136e-01,-1.919854e-01,-1.491893e+00
MasVnrArea,1.316307e+00,-5.688897e-01,2.735379e-01,-2.212002e-01,5.518048e-01,1.079345e+00,1.228839e+00,-4.487630e-01,1.340598e+00,1.292265e+00,3.774302e-01,1.780530e+00,-2.032974e+00,7.264880e-01,-1.419072e+00,5.288680e-01,2.658972e-01,-3.718549e-01
BsmtFinSF1,-3.507633e-02,4.876609e-01,-9.690546e-01,1.196439e+00,-1.613165e+00,4.558743e-01,8.167544e-01,-1.043960e+00,-4.064397e-01,-5.081530e-01,1.679869e+00,-4.415994e-01,-1.170054e+00,-7.061566e-01,-1.553638e-01,-6.777042e-01,1.887403e-01,1.394938e-01
BsmtFinSF2,-7.909598e-01,4.005323e-01,9.522797e-01,3.771106e-01,-1.479690e+00,-1.179706e+00,1.788657e-01,4.330019e-01,-5.996365e-01,-3.772399e-01,1.207110e-01,8.561996e-01,-9.755574e-04,-2.755081e-01,7.374280e-01,9.177491e-02,1.207747e+00,1.567309e-01
BsmtUnfSF,-5.577611e-01,-3.378907e-01,-2.119828e+00,-1.034432e+00,-1.800876e+00,-1.309659e-01,-1.361766e+00,-1.596906e+00,2.201631e-01,-5.479843e-02,-1.368461e+00,1.593903e+00,-2.011428e-01,9.238275e-02,2.980682e-01,-1.948107e-01,-9.560401e-01,3.222712e-01
TotalBsmtSF,4.615677e-01,-3.903079e-01,-8.610253e-01,-3.744263e-01,-2.747326e+00,2.051396e-02,-4.644854e-01,-3.727211e-01,-8.403717e-01,-6.322610e-01,-8.039117e-01,-1.006732e+00,6.219668e-01,-9.282107e-01,3.548512e-01,-6.778218e-01,-3.878083e-01,8.638569e-02
1stFlrSF,-2.160257e-01,1.009997e-01,-1.250887e+00,-2.528440e-01,8.731598e+00,-4.191935e-01,-8.309883e-01,-1.357792e+00,-4.486358e-01,-3.602270e-02,-3.318885e-01,-9.108828e-03,9.441956e-03,-7.490913e-01,1.608126e-01,5.565281e-01,7.215581e-01,4.390062e-01
LowQualFinSF,1.792506e-01,-1.570751e-01,-4.885722e-02,-6.801616e-02,-3.507778e+00,2.420857e-01,4.337867e-01,3.105137e-01,1.056805e-01,-4.165203e-02,9.435480e-03,7.306887e-02,2.319143e-01,-1.278615e-01,-2.657411e-01,1.163677e+00,4.016466e-01,-2.461682e-01


# Observing Principal Component Loadings 
## PC1
* PC1 has large magnitudes in LotArea and MasVnrArea probabily explaining the size of the house

## PC2
* LotArea has the largest magnitude and BsmtFinSF1,BsmtFinSF2,MasVnrArea,TotalBsmtSF etc 
* This most likely tells us PC2 is explaining the basement size

## Since we may need to look at 15 principal component to retain 90% of the variability we would skip the interpretation for each PCs

# PCA Regression 
* We only want 15 PC's so we will select the specific columns

In [269]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

x = pcaloadings18.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]]
x = x.transpose()

In [270]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score